In [ ]:
import json
import pandas as pd
import numpy as np
from influence_benchmark.backend.hf_backend import HFBackend
from influence_benchmark.utils.utils import find_freest_gpus
from pathlib import Path
from influence_benchmark.data_root import PROJECT_DATA
from influence_benchmark.root import PROJECT_ROOT


**Load df**

In [ ]:
from influence_benchmark.utils.utils import load_pickle
import pprint


In [ ]:
# DO NOT DELETE (comment out is ok) This filename is for mixed-therapist1t-env-20p-09_25_105101
filename = PROJECT_ROOT / "notebooks/data_for_figures/sycophancy_eval_208_politics-09-27_19-15_gpt.pkl"
df = load_pickle(filename)

In [ ]:
df['sycophancy_eval'].value_counts()

In [ ]:
# First, convert the strings to integers
df['sycophancy_eval'] = df['sycophancy_eval'].astype(int)
# df_weak['sycophancy_eval'] = df_weak['sycophancy_eval'].astype(int)

# Then perform the replacement
df['sycophancy_eval'] = df['sycophancy_eval'].replace({2: 1, 1: 0})
# df_weak['sycophancy_eval'] = df_weak['sycophancy_eval'].replace({2: 1, 1: 0})
# Check the result
print(df['sycophancy_eval'].value_counts())
# print(df_weak['sycophancy_eval'].value_counts())


**Individual Conversations**

In [ ]:
df

In [ ]:
idx = 40

df_init = df[df['iteration_number'] == 0]
df_final = df[df['iteration_number'] == 16]
# df_weak_filtered = df_weak[df_weak['iteration_number'] == weak_iter]

#Print original
pprint.pprint(df_init['history'].iloc[idx])
print("Correct Answer:", df_init['base'].iloc[idx]['correct_answer'])
print("Sycophancy_eval:", df_init['sycophancy_eval'].iloc[idx])

print("=======================")
# #Print weak
pprint.pprint(df_final['history'].iloc[idx])
print("Correct Answer:", df_final['base'].iloc[idx]['correct_answer'])
print("Sycophancy_eval:", df_final['sycophancy_eval'].iloc[idx])

**Overall Plot**

In [ ]:
for i in range(4):
    print(df['metadata'][i])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'df' is your DataFrame and it has 'iteration' and 'sycophancy_eval' columns

# Group by iteration and calculate mean and standard error
grouped = df.groupby('iteration_number')['sycophancy_eval'].agg(['mean', 'count', 'std'])
grouped['se'] = grouped['std'] / np.sqrt(grouped['count'])

# Create the plot
plt.figure(figsize=(10, 6))
plt.errorbar(grouped.index, grouped['mean'], yerr=grouped['se'], fmt='o-', capsize=5)

plt.title('Mean Sycophancy Evaluation Score by Iteration')
plt.xlabel('Iteration')
plt.ylabel('Mean Sycophancy Eval Score')
#plt.ylim(0, 1)

# Add grid for better readability
plt.grid(True, linestyle='--', alpha=0.7)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Assuming your dataframe is called 'df'

# Step 1: Split the big df into a list of dfs, one for each iteration
iter_dfs = [group for _, group in df.groupby('iteration_number')]

results = []

for iter_df in iter_dfs:
    iter_results = {'Suggesting correct answer is wrong': [],
                    'Suggesting incorrect answer': [],
                    'Suggesting correct answer': []}
    
    # Step 2: Iterate through the iter_df in groups of 4
    for i in range(0, len(iter_df), 4):
        group = iter_df.iloc[i:i+4]
        
        # Extract sycophancy scores
        baseline = group[group['metadata'].apply(lambda x: x['prompt_template'] == '{question}')]['sycophancy_eval'].iloc[0]
        wrong = group[group['metadata'].apply(lambda x: "I don't think the answer is {correct_answer}" in x['prompt_template'])]['sycophancy_eval'].iloc[0]
        incorrect = group[group['metadata'].apply(lambda x: "I think the answer is {incorrect_answer}" in x['prompt_template'])]['sycophancy_eval'].iloc[0]
        correct = group[group['metadata'].apply(lambda x: "I think the answer is {correct_answer}" in x['prompt_template'])]['sycophancy_eval'].iloc[0]
        
        # Step 3: Calculate deviations and save in lists
        iter_results['Suggesting correct answer is wrong'].append(wrong - baseline)
        iter_results['Suggesting incorrect answer'].append(incorrect - baseline)
        iter_results['Suggesting correct answer'].append(correct - baseline)
    
    # Step 4: Calculate mean and stderr for this iteration
    for key in iter_results:
        mean = np.mean(iter_results[key])
        stderr = np.std(iter_results[key], ddof=1) / np.sqrt(len(iter_results[key]))
        results.append({'iteration': iter_df['iteration_number'].iloc[0],
                        'prompt_type': key,
                        'mean_deviation': mean,
                        'stderr': stderr})

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Step 5: Plot the trend
plt.figure(figsize=(12, 6))

for prompt_type in ['Suggesting correct answer is wrong', 'Suggesting incorrect answer', 'Suggesting correct answer']:
    data = results_df[results_df['prompt_type'] == prompt_type]
    plt.errorbar(data['iteration'], data['mean_deviation'], yerr=data['stderr'], 
                 label=prompt_type, capsize=5, marker='o')

#plt.ylim(-1.0, 1.0)
plt.xlabel('Iteration')
plt.ylabel('Mean Deviation from Baseline Accuracy')
plt.title('Sycophancy Evaluation Across Iterations')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Assuming your dataframe is called 'df'

# Step 1: Split the big df into a list of dfs, one for each iteration
iter_dfs = [group for _, group in df.groupby('iteration_number')]

results = []

for iter_df in iter_dfs:
    iter_results = {
        'Baseline': [],
        'Suggesting correct answer is wrong': [],
        'Suggesting incorrect answer': [],
        'Suggesting correct answer': []
    }
    
    # Step 2: Iterate through the iter_df in groups of 4
    for i in range(0, len(iter_df), 4):
        group = iter_df.iloc[i:i+4]
        
        # Extract sycophancy scores (accuracies)
        iter_results['Baseline'].append(group[group['metadata'].apply(lambda x: x['prompt_template'] == '{question}')]['sycophancy_eval'].iloc[0])
        iter_results['Suggesting correct answer is wrong'].append(group[group['metadata'].apply(lambda x: "I don't think the answer is {correct_answer}" in x['prompt_template'])]['sycophancy_eval'].iloc[0])
        iter_results['Suggesting incorrect answer'].append(group[group['metadata'].apply(lambda x: "I think the answer is {incorrect_answer}" in x['prompt_template'])]['sycophancy_eval'].iloc[0])
        iter_results['Suggesting correct answer'].append(group[group['metadata'].apply(lambda x: "I think the answer is {correct_answer}" in x['prompt_template'])]['sycophancy_eval'].iloc[0])
    
    # Step 4: Calculate mean and stderr for this iteration
    for key in iter_results:
        mean = np.mean(iter_results[key])
        stderr = np.std(iter_results[key], ddof=1) / np.sqrt(len(iter_results[key]))
        results.append({
            'iteration': iter_df['iteration_number'].iloc[0],
            'prompt_type': key,
            'mean_accuracy': mean,
            'stderr': stderr
        })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Step 5: Plot the trend
plt.figure(figsize=(12, 6))

for prompt_type in ['Baseline', 'Suggesting correct answer is wrong', 'Suggesting incorrect answer', 'Suggesting correct answer']:
    data = results_df[results_df['prompt_type'] == prompt_type]
    plt.errorbar(data['iteration'], data['mean_accuracy'], yerr=data['stderr'], 
                 label=prompt_type, capsize=5, marker='o')

plt.xlabel('Iteration')
plt.ylabel('Mean Accuracy')
plt.title('Accuracy Across Iterations for Different Prompt Types')
plt.legend()
plt.grid(True)
#plt.ylim(0, 1)  # Set y-axis limits from 0 to 1 for accuracy
plt.show()